In [125]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import filter, col, first, round, concat, lit, when, to_date, trim
from pyspark.sql.window import Window
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

# conf = SparkConf()

# conf.setMaster("local").setAppName("My app")

# export SPARK_CLASSPATH=/path/to/downloaded/jar
# spark-submit --driver-class-path /path/to/downloaded/jar script.py

# bash:  export SPARK_CLASSPATH = C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar
# powershell: $env:SPARK_CLASSPATH = "C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar"
# $ pyspark
# spark-submit --jars "C:/Users/aiden2.kim/Desktop/Aiden-Projects/vacuum-fastapi-dash-etl-app/postgresql-42.6.0.jar" api/rotuers/crdi.py
# bin\pyspark --driver-class-path r"C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar" --jars r"C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar"


spark = SparkSession.builder \
    .appName('crdi-transform') \
    .config("spark.driver.extraClassPath", "C:/Users/aiden2.kim/Desktop/Aiden-Projects/vacuum-fastapi-dash-etl-app/postgresql-42.6.0.jar") \
    .getOrCreate()


# sdf = spark.read.csv('CRDI Appliances Test Insights Stick Vacuums Data.csv', header=True, inferSchema=True)
# sdf.printSchema()
# sdf.show()

In [245]:
# spark.stop()

In [307]:
df_input = spark.read.csv('CRDI Appliances Test Insights Stick Vacuums Data.csv', header=True, inferSchema=True)
df_input.printSchema()
# df_input.show()

root
 |-- category_name: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- rating_category_cro: string (nullable = true)
 |-- test_group: string (nullable = true)
 |-- test_measure: string (nullable = true)
 |-- run_id: string (nullable = true)
 |-- score_value: string (nullable = true)
 |-- units: string (nullable = true)
 |-- parent_brand_name: string (nullable = true)
 |-- parent_model_name: string (nullable = true)



In [308]:
ver_date = '2023-07-18'
# create a new column as category_id
df_input = df_input.withColumn('ver_date', lit(ver_date))\
                    .withColumn('test_category_id', lit(4))\
                    .withColumn('vac_type_id', when(col('category_name').like('%Cordless%'), 1).when(col('category_name').like('%Corded%'), 3))\
                    .withColumn('test_target_id', when(col('rating_category_cro').like('%Bare%'), 1).when(col('rating_category_cro').like('%Carpet%'), 2)\
                                .when(col('rating_category_cro').like('%Edges%'), 3).when(col('rating_category_cro').like('%Pet%'), 4)\
                                .when(col('rating_category_cro').like('%Noise%'), 5).when(col('rating_category_cro').like('%Convenience%'), 6)\
                                .when(col('rating_category_cro').like('%Spec%'), 7).when(col('rating_category_cro').like('%Clean%'), 8)\
                                .when(col('rating_category_cro').like('%score%'), 9).when(col('rating_category_cro').like('%Charge%'), 6) 
                               )\
                    .withColumn('test_group', when(col('test_measure') == 'Debris type', col('score_value')).otherwise(col('test_group')))\
                    .withColumn('test_group_id', when(col('test_group').like('%Sand%'), 1).when(col('test_group').like('%sand%'), 1)\
                                                .when(col('test_group').like('%Mix%'), 6).when(col('test_group').like('%Rice%'), 2)\
                                                .when(col('test_group').like('%Cheerio%'), 3).when(col('test_group').like('%Edge%'), 1)\
                                                .when(col('test_group').like('%Pet hair%'), 4).when(col('test_group').like('%Noise%'), 5)\
                                                .when(col('test_group').like('%Clearance%'), 7).when(col('test_group').like('%Run %'), 8)\
                                                .when(col('test_group').like('%Charge%'), 8).when(col('test_group').like('%Use force%'), 9)\
                                                .when(col('test_group').like('%capacity%'), 10)
                                                
                                               
                                                        
                                                
                                                
                               
                               )
                    
                    
                               
                



df_input.show()
                                                                                                             
                                                                                                             
                                                                                                    
                    # .withColumn('test_group_id', col('another_column') + 10)\
                    # .withColumn('test_measure_id', col('another_column') + 10)\



+--------------------+----------+----------------+-------------------+---------------+--------------------+------+-----------+-----+-----------------+-----------------+----------+----------------+-----------+--------------+-------------+
|       category_name|brand_name|      model_name|rating_category_cro|     test_group|        test_measure|run_id|score_value|units|parent_brand_name|parent_model_name|  ver_date|test_category_id|vac_type_id|test_target_id|test_group_id|
+--------------------+----------+----------------+-------------------+---------------+--------------------+------+-----------+-----+-----------------+-----------------+----------+----------------+-----------+--------------+-------------+
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|           Rice|         Debris type|     1|       Rice| null|             null|             null|2023-07-18|               4|          1|             1|            2|
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1

In [309]:
df_input.createOrReplaceTempView("input")

In [310]:
spark.sql("""
    SELECT DISTINCT
        test_target_id,
        rating_category_cro,
        test_group_id,
        test_group,
        test_measure,
        score_value,
        run_id
    FROM input
    WHERE test_target_id = 1
   

    """).show(1000)


+--------------+-------------------+-------------+-----------+--------------------+-----------+------+
|test_target_id|rating_category_cro|test_group_id| test_group|        test_measure|score_value|run_id|
+--------------+-------------------+-------------+-----------+--------------------+-----------+------+
|             1|        Bare floors|         null|Bare floors|   Pickup Percentage|         81|     6|
|             1|        Bare floors|         null|Bare floors| Final Vacuum Weight|     3489.2|     5|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     3527.5|     3|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     2518.9|     3|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     3390.4|     2|
|             1|        Bare floors|         null|Bare floors|   Pickup Percentage|         97|     1|
|             1|        Bare floors|         null|Bare floors|   Pickup P

In [311]:
# df_input = df_input.withColumn('run_id', when(col('run_id') == 'NA', 0).otherwise(col('run_id')))

In [312]:


# Define a window specification
window_spec = Window.partitionBy('model_name', 'test_target_id', 'test_group_id', 'run_id')
# Create a new DataFrame with the updated 'test_group' column
df_input = df_input.withColumn('test_group', first('test_group').over(window_spec))\
                    .withColumn('test_group_id', first('test_group_id').over(window_spec))




In [313]:
df_input = df_input.withColumn('test_group_id', when((col('test_target_id') == 6) & (col('test_group_id').isNull()), 20).otherwise(col('test_group_id')))

In [314]:
df_input.createOrReplaceTempView("input")

In [316]:
spark.sql("""
    SELECT DISTINCT
        test_target_id,
        rating_category_cro,
        test_group_id,
        test_group,
        test_measure,
        score_value,
        run_id
    FROM input
    WHERE test_group_id is null

    """).show(1000)



+--------------+-------------------+-------------+---------------+--------------------+-----------+------+
|test_target_id|rating_category_cro|test_group_id|     test_group|        test_measure|score_value|run_id|
+--------------+-------------------+-------------+---------------+--------------------+-----------+------+
|             9|      Overall score|         null|  Overall score|     Clean emissions|          5|    NA|
|             8|    Clean emissions|         null|Clean emissions| Vacuum Accumulation|       56.1|     1|
|             9|      Overall score|         null|  Overall score|               Edges|          3|    NA|
|             7|               Spec|         null|           Spec|    Brushroll Window|         NA|    NA|
|             8|    Clean emissions|         null|Clean emissions| Vacuum Accumulation|     599.29|     1|
|             7|               Spec|         null|           Spec|              Weight|        9.9|    NA|
|             9|      Overall score| 

In [296]:
# Define a window specification
window_spec = Window.partitionBy('model_name', 'test_target_id', 'run_id')
# Create a new DataFrame with the updated 'test_group' column
df_input = df_input.withColumn('test_group', first('test_group').over(window_spec))\
                    .withColumn('test_group_id', first('test_group_id').over(window_spec))


In [297]:
df_input.createOrReplaceTempView("input")

spark.sql("""
    SELECT DISTINCT
        test_target_id,
        rating_category_cro,
        test_group_id,
        test_group,
        test_measure,
        score_value,
        run_id
    FROM input
    WHERE test_target_id = 1

    """).show(1000)


+--------------+-------------------+-------------+-----------+--------------------+-----------+------+
|test_target_id|rating_category_cro|test_group_id| test_group|        test_measure|score_value|run_id|
+--------------+-------------------+-------------+-----------+--------------------+-----------+------+
|             1|        Bare floors|         null|Bare floors|   Pickup Percentage|         81|     6|
|             1|        Bare floors|         null|Bare floors| Final Vacuum Weight|     3489.2|     5|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     3527.5|     3|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     2518.9|     3|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     3390.4|     2|
|             1|        Bare floors|         null|Bare floors|         Debris type|       Rice|     4|
|             1|        Bare floors|         null|Bare floors|   Pickup P

In [231]:
spark.sql("""
    SELECT DISTINCT
        test_target_id,
        rating_category_cro,
        test_group_id,
        test_group,
        test_measure,
        score_value
    FROM result
    WHERE test_target_id = 1
   

    """).show(1000)


+--------------+-------------------+-------------+----------+--------------------+-----------+
|test_target_id|rating_category_cro|test_group_id|test_group|        test_measure|score_value|
+--------------+-------------------+-------------+----------+--------------------+-----------+
|             1|        Bare floors|            3|  Cheerios|   Pickup Percentage|         78|
|             1|        Bare floors|            3|  Cheerios| Final Vacuum Weight|     3382.7|
|             1|        Bare floors|            1|      Sand| Final Vacuum Weight|       3956|
|             1|        Bare floors|            1|      Sand|Initial Vacuum We...|     3916.4|
|             1|        Bare floors|            3|  Cheerios|Initial Vacuum We...|     2146.7|
|             1|        Bare floors|            3|  Cheerios| Final Vacuum Weight|     3382.4|
|             1|        Bare floors|            2|      Rice| Final Vacuum Weight|     2615.8|
|             1|        Bare floors|            3|

In [72]:
spark.sql("""
    SELECT DISTINCT
        test_group,
        test_measure,
        score_value
    FROM input
    WHERE score_value IS NOT NULL
        AND TRY_CAST(score_value AS DOUBLE) IS NULL
        AND TRY_CAST(test_group AS DOUBLE) IS NULL

    """).show(1000)


+------------------+--------------------+------------------+
|        test_group|        test_measure|       score_value|
+------------------+--------------------+------------------+
|       Charge time|         Charge time|      Corded model|
|              Spec|        Brush on/off|               Yes|
|              Spec|See-through Dirt Bin|               Yes|
|              Spec|Battery Status In...|               Yes|
|              Spec|Indicator Light f...|               Yes|
|med sand (current)|         Debris type|med sand (current)|
|              Spec|Easy to Maneuver ...|                No|
|              Spec|         Lightweight|                No|
|              Spec|            Headlamp|                No|
|              Spec|Rotating Brush Ad...|                NA|
|              Spec|               Heavy|               Yes|
|              Rice|         Debris type|              Rice|
|              Spec|Rotating Brush Ad...|                No|
|              Spec|    

In [69]:
df_input = df_input.withColumn('test_group',
    when(col('test_measure') == 'Debris type', col('score_value'))
    .otherwise(when(col('test_group').like('%Run time%'), 10).otherwise(col('test_group')))
)

df_input.createOrReplaceTempView("input")

# spark.sql("""
#     SELECT *
#     FROM input
#     WHERE score_value IS NOT NULL
#         AND test_measure = 'Debris type'

#     """).show(1000)


spark.sql("""
    SELECT DISTINCT
        test_group,
        test_measure,
        score_value
    FROM input
    WHERE score_value IS NOT NULL
        AND TRY_CAST(score_value AS DOUBLE) IS NULL

    """).show(1000)




+------------------+--------------------+------------------+
|        test_group|        test_measure|       score_value|
+------------------+--------------------+------------------+
|       Charge time|         Charge time|      Corded model|
|              Spec|        Brush on/off|               Yes|
|              Spec|See-through Dirt Bin|               Yes|
|              Spec|Battery Status In...|               Yes|
|              Spec|Indicator Light f...|               Yes|
|                10|        Power Supply|            corded|
|                10|Manufacturer Powe...|             High |
|med sand (current)|         Debris type|med sand (current)|
|                10|Manufacturer Powe...|               Eco|
|                10|            Run time|      Corded model|
|              Spec|Easy to Maneuver ...|                No|
|                10|Manufacturer Powe...|        Boost Mode|
|                10|Manufacturer Powe...|      Manual Lv. 1|
|              Spec|    

In [13]:
# connect to db

df_vacuum = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "vacuums") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_vacuum.printSchema()


root
 |-- inv_no: integer (nullable = true)
 |-- vac_type_id: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- product_stage: string (nullable = true)
 |-- color: string (nullable = true)
 |-- dual_nozzle: boolean (nullable = true)
 |-- fluffy_nozzle: boolean (nullable = true)
 |-- self_empty: boolean (nullable = true)
 |-- self_clean: boolean (nullable = true)
 |-- release_date: date (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- image: string (nullable = true)
 |-- owner_id: integer (nullable = true)



In [14]:
df_test = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "tests") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_test.printSchema()

root
 |-- id: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- vac_type_id: integer (nullable = true)
 |-- test_status: string (nullable = true)
 |-- assigned1: integer (nullable = true)
 |-- assigned2: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- last_modified: timestamp (nullable = true)
 |-- due_date: timestamp (nullable = true)
 |-- completion_date: timestamp (nullable = true)
 |-- notes: string (nullable = true)
 |-- owner_id: integer (nullable = true)



In [15]:
df_test_vacuums = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "test_vacuums") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_test_vacuums.printSchema()

root
 |-- id: integer (nullable = true)
 |-- test_id: integer (nullable = true)
 |-- vacuum_inv_no: integer (nullable = true)



In [23]:
df_crdi = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "crdi") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_crdi.printSchema()

root
 |-- id: integer (nullable = true)
 |-- ver_date: timestamp (nullable = true)
 |-- test_category_id: integer (nullable = true)
 |-- vac_type_id: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- test_target_id: integer (nullable = true)
 |-- test_group_id: integer (nullable = true)
 |-- test_measure_id: integer (nullable = true)
 |-- test_measure: string (nullable = true)
 |-- value: string (nullable = true)
 |-- units: string (nullable = true)
 |-- run: integer (nullable = true)
 |-- parent_brand: string (nullable = true)
 |-- parent_model: string (nullable = true)
 |-- last_modified: timestamp (nullable = true)
 |-- owner_id: integer (nullable = true)



In [4]:
# # sdf to sql table
# sdf.createOrReplaceTempView('crdi')

'''
# test_details schema

id	                        int (PK)
test_parent_id	            int (FK to tests.id)
test_category_id	        int (FK to test_categories.id)
vac_type_id	                int (FK to vac_types.id)
test_target_id	            int (FK to test_targets.id)
test_group_id               int (FK to test_groups.id)
test_measure_id	            int (FK to test_measures.id)
test_case                   str
tester                      int (FK to users.id)
inv_no                      int (FK to vacuums.id)
brush_type                  str
test_measure                str
value                       str
units                       str
run                         int
run_date                    timestamp
notes                       str
image                       str
owner_id                    int (FK to users.id)

'''

'''
for CRDI data:

test_category_id = 4
vac_type_id: {'Stick Vacuums (Cordless)': 1, 'Stick Vacuums (Corded)': 3}
test_target_id: rating_category_cro

{
     1 - Bare Floors
     2 - Carpet
     3 - Edges
     4 - Pet hair
     5 - Noise
     6 - Convenience
     7 - Specs
     8 - Clean Emissions
     9 - Overall Score,

}

test_group_id:

{
    1 - Sand
    2 - Rice
    3 - Cheerios
    4 - Strokes
    5 - Clearance
    6 - Dustbin
    7 - Features
    8 - Power
    9 - Usability
    10 - Runtime
    11 - Use-Force
    12 - BK-2230
    13 - Charge Time
    14 - Spec
    15 - Bag
    16 - Vacuum Body
    17 - Score
    18 - Pull Ease
    19 - Push Ease

}

test_measure_id:
{
    0 - Soil Weight
    1 - Weight - Before
    2 - Weight - After
    3 -	Weight - Diff.
    4 - Pickup
    5 - Rug Weight - Before
    6 - Rug Weight - After
    7 -	Rug Weight - Diff.
    8 -	Relative Humidity
    9 - Room  Temperature
    10 - Edges Front 1
    11 - Edges Front 2
    12 - Edges Front 3
    13 - Edges Front 4
    14 - Edges Front 5
    15 - Edges Front 6
    16 - Edges Front Avg.
    17 - Edges Left 1
    18 - Edges Left 2
    19 - Edges Left 3
    20 - Edges Left Avg.
    21 - Edges Right 1
    22 - Edges Right 2
    23 - Edges Right 3
    24 - Edges Right Avg.
    25 - Edges Pickup Left
    26 - Edges Pickup Right
    27 - Removal Strokes
    28 - Clearance
    29 - Clearance 3
    30 - Clearance 4
    31 - Clearance 5
    32 - Clearance 6
    33 - Clearance 7
    34 - Clearance 8
    35 - Clearance Avg.
    36 - Score
    37 - Capacity
    38 - Ease
    39 - Empty Weight
    40 - Full Weight
    41 - Features
    42 - Power
    43 - Charge
    44 - Runtime
    45 - Minutes
    46 - Power Level
    47 - Power Supply
    48 - Power Brush Off
    49 - Seconds
    50 - Pull - Avg. Lb
    51 - Push - Avg. Lb
    52 - Pull - Avg. Vac
    53 - Push - Avg. Vac
    54 - Pull - Avg. Vac Not Attached
    55 - Push - Avg. Vac Not Attached
    56 - Corldess
    57 - Mop
    58 - Weight
    59 - Cord Length
    60 - Self-Stand
    61 - Battery Status Indicator - Percentage
    62 - Brush On/Off
    63 - Auto-Suction Mode
    64 - Power Mode Indicator
    65 - Headlight
    66 - Easy-to-Maneuver
    67 - Head Swivels
    68 - Anti Hair-tangle
    69 - Dust Sensor

}


'''




root
 |-- inv_no: integer (nullable = true)
 |-- vac_type_id: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- product_stage: string (nullable = true)
 |-- color: string (nullable = true)
 |-- dual_nozzle: boolean (nullable = true)
 |-- fluffy_nozzle: boolean (nullable = true)
 |-- self_empty: boolean (nullable = true)
 |-- self_clean: boolean (nullable = true)
 |-- release_date: date (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- image: string (nullable = true)
 |-- owner_id: integer (nullable = true)



In [24]:
# register temp tables

df_vacuum.createOrReplaceTempView("vacuum")
df_test.createOrReplaceTempView("test")
df_test_vacuums.createOrReplaceTempView("test_vacuums")
df_crdi.createOrReplaceTempView("crdi")



In [27]:
spark.sql("""
    SELECT *
    FROM input
    LIMIT 10
    """).show()


+--------------------+----------+----------------+-------------------+-----------+--------------------+------+-----------+-----+-----------------+-----------------+
|       category_name|brand_name|      model_name|rating_category_cro| test_group|        test_measure|run_id|score_value|units|parent_brand_name|parent_model_name|
+--------------------+----------+----------------+-------------------+-----------+--------------------+------+-----------+-----+-----------------+-----------------+
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|Bare floors|         Debris type|     1|       Rice| null|             null|             null|
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|Bare floors|         Debris type|     1|       Rice| null|             null|             null|
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|Bare floors|       Debris Weight|     1|         50| gram|             null|             null|
|Stick Vac

In [28]:
df_input = spark.sql("""
    SELECT * FROM input
    """)

In [30]:
df_input.show()

+--------------------+----------+----------------+-------------------+---------------+--------------------+------+-----------+-----+-----------------+-----------------+
|       category_name|brand_name|      model_name|rating_category_cro|     test_group|        test_measure|run_id|score_value|units|parent_brand_name|parent_model_name|
+--------------------+----------+----------------+-------------------+---------------+--------------------+------+-----------+-----+-----------------+-----------------+
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|    Bare floors|         Debris type|     1|       Rice| null|             null|             null|
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|    Bare floors|         Debris type|     1|       Rice| null|             null|             null|
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|    Bare floors|       Debris Weight|     1|         50| gram|             null|     